In [1]:
import pandas as pd 
import os
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import textwrap
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36"
}

In [3]:
with open("APPI OPENAI.txt") as archivo:
  apikey_openai = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey_openai

In [4]:
distritos_lima = [
    "Ancón",
    "Ate",
    "Barranco",
    "Breña",
    "Carabayllo",
    "Chaclacayo",
    "Chorrillos",
    "Cieneguilla",
    "Comas",
    "El Agustino",
    "Independencia",
    "Jesús María",
    "La Molina",
    "La Victoria",
    "Lince",
    "Los Olivos",
    "Lurigancho",
    "Lurín",
    "Magdalena del Mar",
    "Miraflores",
    "Pachacámac",
    "Pucusana",
    "Pueblo Libre",
    "Puente Piedra",
    "Punta Hermosa",
    "Punta Negra",
    "Rímac",
    "San Bartolo",
    "San Borja",
    "San Isidro",
    "San Juan de Lurigancho",
    "San Juan de Miraflores",
    "San Luis",
    "San Martín de Porres",
    "San Miguel",
    "Santa Anita",
    "Santa María del Mar",
    "Santa Rosa",
    "Santiago de Surco",
    "Surquillo",
    "Villa El Salvador",
    "Villa María del Triunfo"
]


In [5]:
departamentos_peru = [
    "Amazonas",
    "Áncash",
    "Apurímac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao", 
    "Cusco",
    "Huancavelica",
    "Huánuco",
    "Ica",
    "Junín",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martín",
    "Tacna",
    "Tumbes",
    "Ucayali"
]


In [6]:
prompt_pais = ChatPromptTemplate.from_messages([
    ("system", """Tu tarea es identificar el o los países mencionados en una noticia.
Responde únicamente con una lista de nombres de países:
- Si se menciona un solo país, devuélvelo en una lista con un solo elemento.
- Si se mencionan varios países, devuélvelos todos en una lista.
- Si no se menciona ningún país, devuelve una lista vacía.
No des explicaciones ni añadas texto adicional. Solo responde con la lista de países."""),
    ("human", "{input}")
])

In [7]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4")

def pais_detector(texto: str) -> list:
    respuesta = llm.invoke(prompt_pais.format_messages(input=texto)).content.strip().lower()
    return respuesta

def buscar_ordenados(texto, lista):
    texto_lower = texto.lower()
    encontrados = []
    for lugar in lista:
        patron = r'\b' + re.escape(lugar.lower()) + r'\b'
        match = re.search(patron, texto_lower)
        if match:
            encontrados.append((lugar, match.start()))  
    encontrados.sort(key=lambda x: x[1])
    return [nombre for nombre, _ in encontrados]

In [8]:
lista_url = []
lista_titulo = []
lista_texto = []
lista_fecha = []
lista_distrito = []
lista_departamento = []
lista_pais = []

In [9]:
for pagina in tqdm(range(1,10)):
    url = f'https://www.exitosanoticias.pe/search?q=feminicidio&p={pagina}'
    try: 
        html = rq.get(url,headers=headers).content.decode("latin-1")
        soup = bs(html) #creamos nuestro arbol
        enlaces_raw = soup.find_all('h2')
        links = [link.find('a').get('href') for link in enlaces_raw if 'href' in link.find('a').attrs ]
        for noticia in tqdm(links): 
            try:
                soup2 = bs(rq.get(noticia, headers=headers).content.decode("latin-1")) 
                try:
                    titulo = soup2.find_all("h1")[0].text
                    fecha = "/".join(list((re.findall(r"(\d{2})[/-](\d{2})[/-](\d{4})",soup2.find('p',class_= 'miga').text))[0]))
                    body = "".join([p.text for p in soup2.find('article',{'class':'content'}).find_all('p')])
                    try:
                        resumen = soup2.find_all("h2")[0].text
                        body = resumen + ' ' + body
                    except:
                        pass
                    body2 = titulo  + ' ' + body
                    distrito = buscar_ordenados(body2, distritos_lima)
                    departamento = buscar_ordenados(body2, departamentos_peru)
                    pais = pais_detector(body)
                    lista_pais.append(pais)
                    lista_url.append(noticia)
                    lista_titulo.append(titulo)
                    lista_texto.append(body)
                    lista_fecha.append(fecha)
                    lista_departamento.append(departamento)
                    lista_distrito.append(distrito)
                except:
                    print(f'Error al encontrar metadata de la noticia: No se encontro metadata: {noticia}\n')
            except Exception as e:
                print(f"Error al procesar noticia: {noticia}\n{e}")
    except Exception as e:
        print(f"Error al cargar la página {pagina}:\n{e}")

100%|██████████| 10/10 [00:20<00:00,  2.04s/it]
0it [00:00, ?it/s]/9 [02:04<02:23, 28.77s/it]
0it [00:00, ?it/s]/9 [02:05<01:14, 18.60s/it]
0it [00:00, ?it/s]/9 [02:05<00:37, 12.50s/it]
0it [00:00, ?it/s]/9 [02:06<00:17,  8.73s/it]
0it [00:00, ?it/s]/9 [02:07<00:06,  6.17s/it]
100%|██████████| 9/9 [02:08<00:00, 14.30s/it]


In [10]:
df = pd.DataFrame({
    'url': lista_url,
    'titulo': lista_titulo,
    'texto': lista_texto,
    'fecha': lista_fecha,
    'distrito': lista_distrito,
    'departamento': lista_departamento,
    'pais': lista_pais})

In [11]:
df.to_excel('Noticias_Exitosa.xlsx',index=False)